# MSA 2024 Phase 2 - Part 3

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
import torch.nn.functional as F

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Yanfe\OneDrive\Desktop\MSA_Phase2\my_env\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

## 1. Data loading & preprocessing

In [ ]:
# Define the ImprovedCNN model
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

class CIFAR10Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        # Convert all label '10' to '0'
        self.labels_frame['label'] = self.labels_frame['label'].apply(lambda x: x % 10)
        # Ensure file paths are correct and images exist
        self.labels_frame['file_path'] = self.labels_frame.apply(lambda x: os.path.join(self.root_dir, f'image_{x.name}.png'), axis=1)
        self.labels_frame = self.labels_frame[self.labels_frame['file_path'].apply(os.path.exists)].reset_index(drop=True)
        # Ensure there are no NaN labels
        self.labels_frame = self.labels_frame.dropna().reset_index(drop=True)

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = self.labels_frame.iloc[idx, 2]
        image = Image.open(img_name).convert("RGB")
        label = self.labels_frame.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Data preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Data loading
train_csv_path = '/kaggle/input/nzmsa-2024/train.csv'
test_csv_path = '/kaggle/input/nzmsa-2024/sample_submission.csv'
train_images_dir = '/kaggle/input/nzmsa-2024/cifar10_images/train'
test_images_dir = '/kaggle/input/nzmsa-2024/cifar10_images/test'

train_dataset = CIFAR10Dataset(csv_file=train_csv_path, root_dir=train_images_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

test_dataset = CIFAR10Dataset(csv_file=test_csv_path, root_dir=test_images_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Use the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        evaluate(model, test_loader, device)

# Evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print(f'Predicted: {predicted.tolist()}, Labels: {labels.tolist()}')
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# Train the model
train_model(model, train_loader, criterion, optimizer, device, num_epochs=10)

# Final evaluation
test_accuracy = evaluate(model, test_loader, device)


: 